In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
import string
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding, GlobalAveragePooling1D, LSTM, Bidirectional
from keras.optimizers import SGD
import json
import tensorflow as tf
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
data_file = '/content/drive/My Drive/Kaggle practice files/python-project-chatbot-codes/intents.json'


Using TensorFlow backend.


In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
data = pd.read_json(data_file)

In [4]:
data[:]

,tag,patterns,responses,context
0,greeting,"[Hi there, How are you, Is anyone there?, Hey,...","[Hello, thanks for asking, Good to see you aga...",[]
1,goodbye,"[Bye, See you later, Goodbye, Nice chatting to...","[See you!, Have a nice day, Bye! Come back aga...",[]
2,thanks,"[Thanks, Thank you, That's helpful, Awesome, t...","[Happy to help!, Any time!, My pleasure]",[]
3,noanswer,[],"[Sorry, can't understand you, Please give me m...",[]
4,options,"[How you could help me?, What you can do?, Wha...",[I can guide you through Adverse drug reaction...,[]
5,adverse_drug,"[How to check Adverse drug reaction?, Open adv...",[Navigating to Adverse drug reaction module],[]
6,blood_pressure,"[Open blood pressure module, Task related to b...",[Navigating to Blood Pressure module],[]
7,blood_pressure_search,[I want to search for blood pressure result hi...,"[Please provide Patient ID, Patient ID?]",[search_blood_pressure_by_patient_id]
8,search_blood_pressure_by_patient_id,[],[Loading Blood pressure result for Patient],[]
9,pharmacy_search,"[Find me a pharmacy, Find pharmacy, List of ph...",[Please provide pharmacy name],[search_pharmacy_by_name]


In [5]:
lemmatizer = WordNetLemmatizer()

In [6]:
words =[]
classes = []
documents =[]

def preprocess(text):
  punct = ''.join([word.lower() for word in text if word not in string.punctuation])
  token = nltk.word_tokenize(punct)
  lem = [lemmatizer.lemmatize(word) for word in token ]
  return lem


for i,x in zip(data.patterns, data.tag):
  for j in i:
    w = preprocess(j)
    if w not in words:  
      words.extend(w)

    documents.append((w,x))

for i in data['tag']:
  if i not in classes and i not in ['noanswer','search_blood_pressure_by_patient_id','search_pharmacy_by_name','search_hospital_by_params','search_hospital_by_type']:
    classes.append(i)

In [7]:
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
86 unique lemmatized words ['a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'thats', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [8]:
documents[:5]

[(['hi', 'there'], 'greeting'),
 (['how', 'are', 'you'], 'greeting'),
 (['is', 'anyone', 'there'], 'greeting'),
 (['hey'], 'greeting'),
 (['hola'], 'greeting')]

In [9]:
pickle.dump(words, open('ChatWords.pkl','wb'))
pickle.dump(classes, open('ChatClasses.pkl','wb'))

In [10]:
doc_df = pd.DataFrame(documents, columns=['features','labels'])
doc_df[:10]

,features,labels
0,"[hi, there]",greeting
1,"[how, are, you]",greeting
2,"[is, anyone, there]",greeting
3,[hey],greeting
4,[hola],greeting
5,[hello],greeting
6,"[good, day]",greeting
7,[bye],goodbye
8,"[see, you, later]",goodbye
9,[goodbye],goodbye


In [11]:
#  ENCODING TRAINING DATA FEATURES AND LABELS

from sklearn.preprocessing import OneHotEncoder

label_df = pd.DataFrame(doc_df.labels)
ohe = OneHotEncoder()
ohe_array = ohe.fit_transform(label_df)
ohe_labels = ohe_array.toarray()  # LABELS ARRAY
ohe_df= pd.DataFrame(ohe_labels)
ohe_df.columns = ohe.categories_

Training_labels = ohe_df   # TRAINING LABELS

Training_labels[:10]

,adverse_drug,blood_pressure,blood_pressure_search,goodbye,greeting,hospital_search,options,pharmacy_search,thanks
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def test (text):
  return text

tf = CountVectorizer(analyzer=test)
t_fit = tf.fit(doc_df.features)

tf_vec = t_fit.transform(doc_df.features)
tf_features = tf_vec.toarray()   # FEATURES ARRAY
tf_df = pd.DataFrame(tf_features)
tf_df.columns = tf.get_feature_names() 
Training_features = tf_df        # TRAINING FEATURES

Training_features[:3]

,a,adverse,all,anyone,are,awesome,be,behavior,blood,by,bye,can,causing,chatting,check,could,data,day,detail,do,dont,drug,entry,find,for,give,good,goodbye,have,hello,help,helpful,helping,hey,hi,history,hola,hospital,how,i,...,log,looking,lookup,management,me,module,nearby,next,nice,of,offered,open,patient,pharmacy,pressure,provide,reaction,related,result,search,searching,see,show,suitable,support,task,thank,thanks,thats,there,till,time,to,transfer,up,want,what,which,with,you
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [13]:
train_x = np.array(Training_features)
train_y = np.array(Training_labels)

print("Training data created")

print(len(train_y[0]))
print(len(train_x[0]))

Training data created
9
86


## MODEL

In [14]:
model = Sequential()
model.add(Dense(128, input_shape=(86,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               11136     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 585       
Total params: 19,977
Trainable params: 19,977
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
47/47 [==============================] - 0s 6ms/step - loss: 2.2369 - accuracy: 0.1064
Epoch 2/200
47/47 [==============================] - 0s 436us/step - loss: 2.2341 - accuracy: 0.1277
Epoch 3/200
47/47 [==============================] - 0s 406us/step - loss: 2.1359 - accuracy: 0.1915
Epoch 4/200
47/47 [==============================] - 0s 398us/step - loss: 2.0169 - accuracy: 0.2979
Epoch 5/200
47/47 [==============================] - 0s 382us/step - loss: 1.9502 - accuracy: 0.3830
Epoch 6/200
47/47 [==============================] - 0s 342us/step - loss: 1.8392 - accuracy: 0.3617
Epoch 7/200
47/47 [==============================] - 0s 349us/step - loss: 1.7393 - accuracy: 0.4894
Epoch 8/200
47/47 [==============================] - 0s 414us/step - loss: 1.6176 - accuracy: 0.5319
Epoch 9/200
47/47 [==============================] - 0s 379us/step - loss: 1.4544 - accuracy: 0.5957
Epoch 10/200
47/47 [==============================] - 0s 352us/step - loss: 1.3512 - accuracy

In [17]:
model.save('/content/drive/My Drive/Kaggle practice files/Akshay Chatbot/ akshay_chatbot.h5')

## PREDICT

In [18]:
JARVIS = keras.models.load_model('/content/drive/My Drive/Kaggle practice files/Akshay Chatbot/ akshay_chatbot.h5')

In [19]:
def word_vectors(text, words):
  prep = preprocess(text)
  bag =[0]*len(words)
  for se in prep:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1        
  return (np.array(bag))

In [20]:
def talk():
  print (" Hi AKSHAY!!!  I am JARVIS")
  for i in range(0,5):
    inp = input("You:  ")
    p = word_vectors(inp, words)
    res = JARVIS.predict(np.array([p]))[0]
    res = np.argmax(res)
    res_label = classes[res]

    for i in data['tag']:
      if i == res_label:
        respond = data.loc[data['tag'] == i, 'responses']
        for i in respond:
          reply = random.choice(i)


    print(reply)

  


In [21]:
talk() 

 Hi AKSHAY!!!  I am JARVIS
You:  hi
Hello, thanks for asking
You:  blood pressure?
Navigating to Blood Pressure module
You:  
Good to see you again
You:  
Good to see you again
You:  bye
Have a nice day
